In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from data_preprocessing import *
import time

methods = ["SIFT", "KAZE", "AKAZE", "ORB", "BRISK"]

In [ ]:
time_compare = pd.DataFrame(columns=["method", "time"])
for m in methods:
    start = time.time()
    get_keypoints_and_descriptors("/Users/barborakarlikova/FIT/3rd_year/B232/VWM/data/butterflies_all/", 
                                  get_method_according_to_string(m)[0])
    end = time.time()
    time_compare.loc[len(time_compare.index)] = [m, end - start]

In [ ]:
display(time_compare)

In [ ]:
df = pd.read_csv("/Users/barborakarlikova/FIT/3rd_year/B232/VWM/bi-vwm/test_data/Training_set.csv")
df.loc[len(df.index)] = ["Image_3704_upsidedown.jpg", "PAPER KITE"]
df.loc[len(df.index)] = ["Image_4721_zoom.jpg", "CAIRNS BIRDWING"]
df.loc[len(df.index)] = ["Image_5843_right.jpg", "VICEROY"]
df.loc[len(df.index)] = ["Image_5438_fliped.jpg", "CLOUDED SULPHUR"]
df.loc[len(df.index)] = ["Image_3642_binary.jpg", "GREEN CELLED CATTLEHEART"]

In [ ]:
zeros = np.zeros(len(os.listdir("/Users/barborakarlikova/FIT/3rd_year/B232/VWM/bi-vwm/test_data/butterflies_y/")))

In [ ]:
stats = pd.DataFrame(data = {"SIFT": zeros, "KAZE": zeros, "AKAZE": zeros, "ORB":zeros, "BRISK" : zeros} )
for m in methods:
    x = get_keypoints_and_descriptors("/Users/barborakarlikova/FIT/3rd_year/B232/VWM/bi-vwm/test_data/butterflies_x/", get_method_according_to_string(m)[0])
    y = get_keypoints_and_descriptors("/Users/barborakarlikova/FIT/3rd_year/B232/VWM/bi-vwm/test_data/butterflies_y/", get_method_according_to_string(m)[0]) 
    x_lim = len(x)
    y_lim = len(y)
    res = find_similarity(x, y)
    for _, row in res.iterrows():
        for i, (key, value) in enumerate(row["similarities"]):
            if df[df["filename"] == row["image"]]["label"].values[0] == df[df["filename"] == key]["label"].values[0]:
                stats.loc[i, m] += 1
                

In [ ]:
fig, ax = plt.subplots(layout='constrained', figsize=(20,8))
width=0.15
multiplier = 1
colors = ["#024b7e", "#3ebaca","#9cdbea","#e99f8f", "#eada6d" ,]
x = np.arange(len(stats))
for method, color in zip(methods, colors):
    offset = width * multiplier
    rects = ax.bar(x+offset, stats[method].values, width, color=color, label=method)
    #ax.bar_label(rects, padding=3)
    multiplier += 1

ax.grid(True, axis="y")
ax.set_ylabel('number of correct images')
ax.set_xlabel('n-th similar image')
ax.set_ylim(0,x_lim)
ax.set_xlim(0,y_lim)
ax.set_title('Correctness per feature detection algorithms')
ax.set_xticks(x + width, stats.index)
ax.legend(loc='upper center', ncols=10)

plt.show()